In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
import sqlite3
import duckdb
di = data_inserts()

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()
# di.insert_datasets()
# di.insert_years()
di.insert_geo_full()

In [ ]:
di.data.select("c_geographyLink").to_series().to_list()

In [ ]:
base = "http://api.census.gov/data/2017/abscb/geography.json"
# results = requests.get(base).json()
df = pl.DataFrame(requests.get(base).json().get("fips"))
df

In [ ]:
df.select(pl.col("name") + "-" + pl.col("geoLevelDisplay")).to_series().to_list()

In [ ]:
for obs in df.select(pl.col("name") + "-" + pl.col("geoLevelDisplay")).to_series().to_list():
    geo_desc, geo_lv = obs.split("-")
    if di.get_geo_id(geo_lv=geo_lv) == -1:
        di.conn.execute(f"""
            INSERT INTO sqlite_db.geo_table 
                (geo_name, geo_lv) VALUES ('{geo_desc}', '{geo_lv}');
        """)
        print(f"inserted {geo_desc} into db")

    geo_id = di.get_geo_id(geo_lv=geo_lv)
    year_id = di.get_year_id(base.split("/")[4])
    dataset_id = di.get_dataset_id(base.split("/")[5])

    if di.check_geo_interm_id(dataset_id=dataset_id, geo_id=geo_id, year_id=year_id):
        continue
    else:
        di.insert_geo_interm(dataset_id=dataset_id, geo_id=geo_id, year_id=year_id)
    

In [ ]:
geo_desc = "us"
geo_lv = "010"
geo_id = di.get_geo_id(geo_lv=geo_lv)

year = base.split("/")[4]
year_id = di.get_year_id(base.split("/")[4])

dataset = base.split("/")[5]
dataset_id = di.get_dataset_id(base.split("/")[5])

query = f"INSERT INTO sqlite_db.geo_interm (dataset_id, geo_id, year_id) VALUES ({dataset_id}, {geo_id}, {year_id});"
query2 = f"SELECT * FROM sqlite_db.geo_interm;"

di.conn.execute(query=query2).pl()

In [ ]:
di.conn.execute("SELECT * FROM sqlite_db.geo_table;").pl()

In [ ]:
df.select("geoLevelDisplay").to_series().to_list()